In [1]:
import pandas as pd
import math
from sklearn.metrics import confusion_matrix, roc_auc_score
from sklearn import metrics
import numpy as np
import scipy.optimize as optimize

In [2]:
usa_ = pd.read_csv('usa_.csv')

Homogeneous process

In [3]:
test = pd.DataFrame()
train = pd.DataFrame()

In [4]:
for name in usa_.columns:
    test[name] = []
    train[name] = []

In [5]:
for acc in usa_['AccountId'].unique():
    df_tmp = usa_.loc[usa_['AccountId'] == acc]
    li = df_tmp['TransactionId'].to_list()
    a = int(len(li) // 8 + 1)
    test = test.append(df_tmp.tail(a), ignore_index = True)
    train = train.append(df_tmp.head(len(li) - a), ignore_index = True)

In [6]:
lyambda = {}

In [7]:
for card in train['AccountId'].unique():
    df_tmp = train.loc[(train['AccountId'] == card) & (train['IsFraud'] == 1)]
    li = df_tmp['TD'].to_list()
    if (len(li) == 0):
        lyambda[card] = 0
    else:
        lyambda[card] = float( len(li) / li[-1] )

In [8]:
label = {}
for card in test['AccountId'].unique():
    df_tmp = test.loc[test['AccountId'] == card]
    trans = {}
    for transaction in df_tmp['TransactionId'].to_list():
        trans[transaction] = df_tmp.loc[df_tmp['TransactionId'] == transaction]['IsFraud'].to_list()[0]
    label[card] = trans

In [9]:
pred = {}

for card in test['AccountId'].unique():
    df_tmp = test.loc[test['AccountId'] == card]
    T = train.loc[train['AccountId'] == card]['TD'].to_list()[-1]
    y = {}
    for transaction in df_tmp['TransactionId'].to_list():
       # if (transaction == df_tmp['TransactionId'].to_list()[0]):
        y[transaction] = 1 - math.exp(-lyambda[card] * (df_tmp.loc[df_tmp['TransactionId'] == transaction]['TD'].to_list()[0] - T))
        T = df_tmp.loc[df_tmp['TransactionId'] == transaction]['TD'].to_list()[0]
    pred[card] = y

In [10]:
label_li = []
pred_li = []

In [11]:
for acc in test['AccountId'].unique():
    df_tmp = test.loc[test['AccountId'] == acc]
    for transaction in df_tmp['TransactionId'].to_list():
        label_li.append(label[acc][transaction])
        pred_li.append(pred[acc][transaction])

In [12]:
for i in range(len(pred_li)):
    if (pred_li[i] >= 0.5):
        pred_li[i] = 1
    else:
        pred_li[i] = 0

In [13]:
from auc import comp_auc

print(f"AUC_test from HomoModel: {comp_auc(label_li, pred_li)}")

AUC_test from HomoModel: 0.748951


Non-homogeneous process

In [14]:
test = pd.DataFrame()
train = pd.DataFrame()

for name in usa_.columns:
    test[name] = []
    train[name] = []
    
for acc in usa_['AccountId'].unique():
    df_tmp = usa_.loc[usa_['AccountId'] == acc]
    li = df_tmp['TransactionId'].to_list()
    a = int(len(li) // 8 + 1)
    test = test.append(df_tmp.tail(a), ignore_index = True)
    train = train.append(df_tmp.head(len(li) - a), ignore_index = True)

#### lyambda(t) = at + b

In [15]:
a = {}
b = {}

In [16]:
def func(w):
    s = 0
    td = li
    T = li[-1]
    for i in range(len(td)):
        s += math.log(w[0]*td[i] + w[1]) - w[0] * T ** 2 / 2 - w[1] * T
    return -s

In [17]:
for acc in train['AccountId'].unique():
    df_tmp = train.loc[(train['AccountId'] == acc) & (train['IsFraud'] == 1)]
    li = df_tmp['TransactionId'].to_list()
    if len(li) == 0:
        a[acc] = 0
        b[acc] = 0
    else:
        T = li[-1]
        max = optimize.minimize(func, np.array([100, 10]), method = 'L-BFGS-B', bounds = ((1, 100), (1, 10)))
        a[acc] = max.x[0]
        b[acc] = max.x[1]

In [18]:
label = {}
for card in test['AccountId'].unique():
    df_tmp = test.loc[test['AccountId'] == card]
    trans = {}
    for transaction in df_tmp['TransactionId'].to_list():
        trans[transaction] = df_tmp.loc[df_tmp['TransactionId'] == transaction]['IsFraud'].to_list()[0]
    label[card] = trans

In [19]:
pred = {}

for card in test['AccountId'].unique():
    df_tmp = test.loc[test['AccountId'] == card]
    T = train.loc[train['AccountId'] == card]['TD'].to_list()[-1]
    y = {}
    for transaction in df_tmp['TransactionId'].to_list():
        now = df_tmp.loc[df_tmp['TransactionId'] == transaction]['TD'].to_list()[0]
        y[transaction] = 1 - math.exp( a[card] / 2 *(T ** 2 - now ** 2) - b[card] * (now - T) )
        T = now
    pred[card] = y

In [20]:
label_li = []
pred_li = []

for acc in test['AccountId'].unique():
    df_tmp = test.loc[test['AccountId'] == acc]
    for transaction in df_tmp['TransactionId'].to_list():
        label_li.append(label[acc][transaction])
        pred_li.append(pred[acc][transaction])

for i in range(len(pred_li)):
    if (pred_li[i] >= 0.5):
        pred_li[i] = 1
    else:
        pred_li[i] = 0

In [21]:
print(f"AUC_test from LinearModel: {comp_auc(label_li, pred_li)}")

AUC_test from LinearModel: 0.786327


#### lyambda(t) = at^2 + bt + c

In [23]:
a = {}
b = {}
c = {}

In [24]:
def func(w):
    s = 0
    td = li
    T = li[-1]
    for i in range(len(td)):
        s += math.log(w[0]*td[i] ** 2 + w[1]*td[i] + w[2]) - w[0] * T ** 3 / 3 - w[1] * T ** 2 / 2 - w[2] * T
    return -s

In [25]:
for acc in train['AccountId'].unique():
    df_tmp = train.loc[(train['AccountId'] == acc) & (train['IsFraud'] == 1)]
    li = df_tmp['TransactionId'].to_list()
    if len(li) == 0:
        a[acc] = 0
        b[acc] = 0
        c[acc] = 0
    else:
        T = li[-1]
        max = optimize.minimize(func, np.array([100, 10, 15]), method = 'L-BFGS-B', bounds = ((1, 100), (1, 10), (1, 15)))
        a[acc] = max.x[0]
        b[acc] = max.x[1]
        c[acc] = max.x[2]

In [26]:
label = {}
for card in test['AccountId'].unique():
    df_tmp = test.loc[test['AccountId'] == card]
    trans = {}
    for transaction in df_tmp['TransactionId'].to_list():
        trans[transaction] = df_tmp.loc[df_tmp['TransactionId'] == transaction]['IsFraud'].to_list()[0]
    label[card] = trans

In [27]:
pred = {}

for card in test['AccountId'].unique():
    df_tmp = test.loc[test['AccountId'] == card]
    T = train.loc[train['AccountId'] == card]['TD'].to_list()[-1]
    y = {}
    for transaction in df_tmp['TransactionId'].to_list():
        now = df_tmp.loc[df_tmp['TransactionId'] == transaction]['TD'].to_list()[0]
        y[transaction] = 1 - math.exp( a[card] / 3 *(T ** 3 - now ** 3) + b[card] / 2 * (T ** 2 - now ** 2) + c[card] * (T - now) )
        T = now
    pred[card] = y

In [28]:
label_li = []
pred_li = []

for acc in test['AccountId'].unique():
    df_tmp = test.loc[test['AccountId'] == acc]
    for transaction in df_tmp['TransactionId'].to_list():
        label_li.append(label[acc][transaction])
        pred_li.append(pred[acc][transaction])

for i in range(len(pred_li)):
    if (pred_li[i] >= 0.5):
        pred_li[i] = 1
    else:
        pred_li[i] = 0

In [29]:
print(f"AUC_test from LinearModel: {comp_auc(label_li, pred_li)}")

AUC_test from LinearModel: 0.769392
